In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import datetime as dt
import os
import warnings 
warnings.filterwarnings("ignore")

In [2]:
data_1hr=pd.read_csv("NSE_BAJFINANCE1!, 60_1dc31.csv")
data_1hr

,time,open,high,low,close
0,1651466700,6649.35,6665.10,6614.65,6626.60
1,1651470300,6627.00,6659.75,6622.15,6638.00
2,1651473900,6638.75,6653.05,6623.00,6644.35
3,1651477500,6644.15,6647.05,6610.05,6628.00
4,1651481100,6628.00,6669.00,6616.30,6660.00
...,...,...,...,...,...
295,1656567900,5447.50,5449.40,5410.00,5414.50
296,1656571500,5416.60,5430.90,5385.00,5430.90
297,1656575100,5431.00,5464.00,5425.15,5448.35
298,1656578700,5448.35,5450.55,5400.00,5408.00


In [3]:
data_1hr.time = data_1hr.time.apply(lambda d: \
                                  dt.datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d %H:%M:%S'))

In [4]:
data_1hr.head()

,time,open,high,low,close
0,2022-05-02 10:15:00,6649.35,6665.10,6614.65,6626.60
1,2022-05-02 11:15:00,6627.00,6659.75,6622.15,6638.00
2,2022-05-02 12:15:00,6638.75,6653.05,6623.00,6644.35
3,2022-05-02 13:15:00,6644.15,6647.05,6610.05,6628.00
4,2022-05-02 14:15:00,6628.00,6669.00,6616.30,6660.00


In [5]:
data_15=pd.read_csv("NSE_BAJFINANCE1!, 15_45244.csv", parse_dates=True, index_col=0)
data_15

,open,high,low,close
time,,,,
1655113500,5340.05,5361.45,5340.05,5360.05
1655178300,5330.00,5387.05,5281.50,5285.00
1655179200,5284.50,5394.40,5284.50,5394.40
1655180100,5396.00,5428.00,5389.20,5404.10
1655181000,5403.90,5413.65,5372.80,5385.95
...,...,...,...,...
1656492300,5523.00,5531.95,5518.05,5524.20
1656493200,5524.20,5526.30,5501.05,5509.20
1656494100,5509.20,5512.60,5497.20,5507.50


In [11]:
data_15.index = pd.to_datetime(data_15.index)
data_15

,open,high,low,close
time,,,,
1970-01-01 00:00:01.655113500,5340.05,5361.45,5340.05,5360.05
1970-01-01 00:00:01.655178300,5330.00,5387.05,5281.50,5285.00
1970-01-01 00:00:01.655179200,5284.50,5394.40,5284.50,5394.40
1970-01-01 00:00:01.655180100,5396.00,5428.00,5389.20,5404.10
1970-01-01 00:00:01.655181000,5403.90,5413.65,5372.80,5385.95
...,...,...,...,...
1970-01-01 00:00:01.656492300,5523.00,5531.95,5518.05,5524.20
1970-01-01 00:00:01.656493200,5524.20,5526.30,5501.05,5509.20
1970-01-01 00:00:01.656494100,5509.20,5512.60,5497.20,5507.50


In [5]:
data_1hr['SMA_10']=data_1hr['high'].rolling(window = 10).mean()
data_1hr['SMA_8']=data_1hr['low'].rolling(window=8).mean()
data_1hr['EMA_50']=data_1hr['close'].ewm(span=50, adjust=False).mean()
data_1hr['Prev_low1']=data_1hr['low'].shift(1)
data_1hr['Prev_low2']=data_1hr['low'].shift(2)

In [6]:

def williams_ad(data, high_col='high', low_col='low', close_col='close'):
    data['williams_ad'] = 0.
    
    for index,row in data.iterrows():
        if index > 0:
            prev_value = data.at[index-1, 'williams_ad']
            prev_close = data.at[index-1, close_col]
            if row[close_col] > prev_close:
                ad = row[close_col] - min(prev_close, row[low_col])
            elif row[close_col] < prev_close:
                ad = row[close_col] - max(prev_close, row[high_col])
            else:
                ad = 0                                                                                          
            data.at[index,'williams_ad'] = (ad+prev_value)
        
    return data


In [7]:
t_1=williams_ad(data_1hr)
t_1['rolling_william']=t_1['williams_ad'].rolling(window=57).mean()
t_1

,time,open,high,low,close,SMA_10,SMA_8,EMA_50,Prev_low1,Prev_low2,williams_ad,rolling_william
0,2022-05-02 10:15:00,6649.35,6665.10,6614.65,6626.60,NaN,NaN,6626.600000,NaN,NaN,0.00,NaN
1,2022-05-02 11:15:00,6627.00,6659.75,6622.15,6638.00,NaN,NaN,6627.047059,6614.65,NaN,15.85,NaN
2,2022-05-02 12:15:00,6638.75,6653.05,6623.00,6644.35,NaN,NaN,6627.725606,6622.15,6614.65,37.20,NaN
3,2022-05-02 13:15:00,6644.15,6647.05,6610.05,6628.00,NaN,NaN,6627.736366,6623.00,6622.15,18.15,NaN
4,2022-05-02 14:15:00,6628.00,6669.00,6616.30,6660.00,NaN,NaN,6629.001607,6610.05,6623.00,61.85,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2022-06-30 11:15:00,5447.50,5449.40,5410.00,5414.50,5510.385,5472.43750,5511.269586,5443.25,5475.30,-1406.35,-1271.572807
296,2022-06-30 12:15:00,5416.60,5430.90,5385.00,5430.90,5503.475,5458.91250,5508.117838,5410.00,5443.25,-1360.45,-1272.356140
297,2022-06-30 13:15:00,5431.00,5464.00,5425.15,5448.35,5497.945,5450.41875,5505.774001,5385.00,5410.00,-1337.25,-1271.862281
298,2022-06-30 14:15:00,5448.35,5450.55,5400.00,5408.00,5490.525,5437.28125,5501.939726,5425.15,5385.00,-1379.80,-1271.839474


In [8]:
t_1["Signal"] = np.where(((t_1['Prev_low1']>t_1['SMA_10'])\
                        & (t_1['Prev_low2']>t_1['SMA_10'])\
                        & (t_1['close']>t_1['EMA_50'])\
                        & (t_1['williams_ad']>t_1['rolling_william'])),1,0)
                

In [9]:
t_1.to_excel('SignalFile.xlsx',index=None)

In [10]:
######################### Main loop #############################
TradeBook = pd.DataFrame(columns=['EntryDate','EntryPrice','Position','ExitDate','ExitPrice','PNL'\
                                 ,'Reason'])
OpenPosDict=dict()
OpenPosDict = {'EntryDate':None,'EntryPrice':None,'Position':None,'ExitDate':None,'ExitPrice':\
                  None,'PNL':None,'Reason':None}
for i in range(len(t_1)):
    
    ###### Check the condition to take new Position ##########################
    if ((t_1['Signal'][i] ==1)& (OpenPosDict['Position'] is None)):
        OpenPosDict['EntryDate']=t_1['time'][i]
        OpenPosDict['EntryPrice']=t_1['SMA_8'][i]
        OpenPosDict['Position']="Long"
         ######## position is already there ###################
    elif OpenPosDict['Position'] == "Long":
        
            ####### check for stop trigger ########################
        if (((t_1['close'][i]<t_1['EMA_50'][i])) or \
           (t_1['close'][i]<(OpenPosDict['EntryPrice'] * 0.97))):
                ###### stop trigger #########
                OpenPosDict['ExitDate']=t_1['time'][i]
                OpenPosDict['ExitPrice']=t_1['close'][i]
                OpenPosDict['PNL']=(OpenPosDict['ExitPrice']-OpenPosDict['EntryPrice'])
                OpenPosDict['Reason']="Stoploss"
                TradeBook.loc[len(TradeBook)] = [OpenPosDict['EntryDate'],OpenPosDict['EntryPrice'],\
                                                OpenPosDict['Position'],\
                                                OpenPosDict['ExitDate'],\
                                                OpenPosDict['ExitPrice'],\
                                                OpenPosDict['PNL'],OpenPosDict['Reason']]
                # Reset the OpenPosition Dict
                OpenPosDict = OpenPosDict.fromkeys(OpenPosDict, None)
            ####### check for target trigger ########
        elif (t_1['close'][i]>(OpenPosDict['EntryPrice']* 1.03)):
                ####### target trigger #######
                OpenPosDict['ExitDate']=t_1['time'][i]
                OpenPosDict['ExitPrice']=t_1['close'][i]
                OpenPosDict['PNL']=(OpenPosDict['ExitPrice']-OpenPosDict['EntryPrice'])
                OpenPosDict['Reason']="Takeprofit"
                ###### record the trade ################
                TradeBook.loc[len(TradeBook)] = [OpenPosDict['EntryDate'],OpenPosDict['EntryPrice'],\
                                                OpenPosDict['Position'],\
                                                OpenPosDict['ExitDate'],\
                                                OpenPosDict['ExitPrice'],\
                                                OpenPosDict['PNL'],OpenPosDict['Reason']]
                # Reset the OpenPosition Dict
                OpenPosDict = OpenPosDict.fromkeys(OpenPosDict, None)
       

In [11]:
TradeBook

,EntryDate,EntryPrice,Position,ExitDate,ExitPrice,PNL,Reason
0,2022-05-18 09:15:00,5754.94375,Long,2022-05-18 11:15:00,5829.40,74.45625,Stoploss
1,2022-05-27 12:15:00,5845.45000,Long,2022-05-30 09:15:00,6055.00,209.55000,Takeprofit
2,2022-06-03 11:15:00,6053.72500,Long,2022-06-06 09:15:00,5912.50,-141.22500,Stoploss
3,2022-06-24 15:15:00,5495.87500,Long,2022-06-28 09:15:00,5518.85,22.97500,Stoploss


In [12]:
TradeBook.to_excel('TradeBook.xlsx',index=None)